In [ ]:
!pip install ktrain

In [2]:
import matplotlib.pyplot as plt
import tensorflow as tf
from ktrain import text
import pandas as pd
import numpy as np
import ktrain
import nltk
import re

In [6]:
train = pd.read_csv("https://raw.githubusercontent.com/ishantjuyal/Emotions-Detection/main/Data/Emotion_final.csv")

In [7]:
print("Shape of train is", train.shape)
train.head()

Shape of train is (18000, 2)


,text,emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [8]:
(X_train, y_train), (X_test, y_test), preprocess = text.texts_from_df(train_df= train, 
                                                                      text_column = 'text',
                                                                      label_columns = 'emotion',
                                                                      maxlen = 50,
                                                                      preprocess_mode = 'bert')

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [9]:
model = text.text_classifier(name= 'bert',
                             train_data= (X_train, y_train),
                             preproc = preprocess)

Is Multi-Label? False
maxlen is 50
done.


In [10]:
# Get Learning Rate
learner = ktrain.get_learner(model = model,
                             train_data = (X_train, y_train),
                             val_data = (X_test, y_test),
                             batch_size = 32)

In [11]:
learner.fit_onecycle(lr = 2e-5, epochs = 1)



begin training using onecycle policy with max lr of 2e-05...
507/507 [==============================] - 443s 873ms/step - loss: 0.7108 - accuracy: 0.7372 - val_loss: 0.1976 - val_accuracy: 0.9167


In [12]:
predictor = ktrain.get_predictor(learner.model, preproc= preprocess)

In [13]:
test = get_data2('test.txt')
test.head()

,text,emotion
0,im feeling rather rotten so im not very ambiti...,sadness
1,im updating my blog because i feel shitty,sadness
2,i never make her separate from me because i do...,sadness
3,i left with my bouquet of red and yellow tulip...,joy
4,i was feeling a little vain when i did this one,sadness


In [14]:
y_t = np.array(test['emotion'])
y_t

array(['sadness', 'sadness', 'sadness', ..., 'joy', 'joy', 'fear'],
      dtype=object)

In [15]:
data = np.array(test['text'])

In [16]:
y_pred = predictor.predict(data)

In [17]:
y_pred = np.array(y_pred)
y_pred

array(['sadness', 'sadness', 'sadness', ..., 'joy', 'joy', 'fear'],
      dtype='<U8')

In [18]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
print("Accuracy score for test data is:")
print(accuracy_score(y_t, y_pred))
print("\nConfusion Matrix for test data is:")
print(confusion_matrix(y_t, y_pred))

Accuracy score for test data is:
0.9205

Confusion Matrix for test data is:
[[251  10   4   1   9   0]
 [  3 205   1   0   5  10]
 [  3   0 650  34   7   1]
 [  2   0  15 141   1   0]
 [ 13   7   8   4 549   0]
 [  0  14   4   0   3  45]]


In [19]:
data = ["I have really become used to this cycle where I stay happy for 14-15 days and then spend the next 5-6 days feeling worthless and feeling like doing nothing at all.These emotions are not clear, they are not the primary emotions that I face because I don’t have a clear picture of what I am feeling.I haven’t felt love, I have stopped feeling any kind of emotion and I have become numb, completely numb.This feeling of emptiness, like a void which cannot be filled now.Even when the lockdown is not there, I feel like I am in lockdown.I guess this 3 weeks cycle is my routine now until things get better and life gets back to what it was."]

In [20]:
predictor.predict(data)

['sadness']

In [21]:
data = ["I am pretty sad and I don't have anyone to talk to.", 
        "I am great actually. Very happy with my life for now",
        "I am really scared of what is going to happen to me now"]

In [22]:
predictor.predict(data)

['sadness', 'joy', 'fear']

In [23]:
def detect(sentence):
    data = [sentence]
    emotion = predictor.predict(data)
    print('You are feeling ' + emotion[0])

In [24]:
detect("I have really become used to this cycle where I stay happy for 14-15 days and then spend the next 5-6 days feeling worthless and feeling like doing nothing at all.These emotions are not clear, they are not the primary emotions that I face because I don’t have a clear picture of what I am feeling.I haven’t felt love, I have stopped feeling any kind of emotion and I have become numb, completely numb.This feeling of emptiness, like a void which cannot be filled now.Even when the lockdown is not there, I feel like I am in lockdown.I guess this 3 weeks cycle is my routine now until things get better and life gets back to what it was.")

You are feeling sadness
